# Basics
The basic notebook just imports the required dependencies and sets the ports and addresses correctly to be accessible outside of the Docker container. It then creates a link for viewing the data

In [1]:
from __future__ import print_function
from neuroglancer.server import global_server_args

# setup the binding so it is visible outside the container
PORT = 8989
global_server_args['bind_port'] = PORT
global_server_args['bind_address'] = '00.0.0.0' # TODO: the server renames addresses of 0.0.0.0 and they become unusable
import neuroglancer
import numpy as np

# Synthetic Skeleton Data
A simple skeleton data source

In [2]:
voxel_size = np.array([10, 10, 10])
shape = (100, 100, 100)
segmentation = np.arange(np.prod(shape), dtype=np.uint32).reshape(shape)

class SkeletonSource(neuroglancer.skeleton.SkeletonSource):
    def __init__(self):
        super(SkeletonSource, self).__init__()
        self.vertex_attributes['affinity'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )
        self.vertex_attributes['affinity2'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )

    def get_skeleton(self, i):
        pos = np.unravel_index(i, shape, order='C')[::-1]
        vertex_positions = [pos * voxel_size, pos * voxel_size + np.random.randn(3) * 300]
        edges = [0, 1]
        return neuroglancer.skeleton.Skeleton(
            vertex_positions=vertex_positions,
            edges=edges,
            vertex_attributes=dict(affinity=np.random.rand(2), affinity2=np.random.rand(2)))

# Create the viewer
Here the viewer is created with a simple shader to make the rendering colorful

In [3]:
neuroglancer.set_static_content_source(url='https://neuroglancer-demo.appspot.com')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers.append(
        name='a',
        layer=neuroglancer.SegmentationLayer(
            source=neuroglancer.LocalVolume(
                data=segmentation,
                voxel_size=voxel_size,
                skeletons=SkeletonSource(),
            ),
            skeleton_shader='void main() { emitRGB(colormapJet(affinity)); }',
            selected_alpha=0,
            not_selected_alpha=0,
        ))

# Show the Results
Here when we show the viewer we get a link, altnatively the results can be shown using the IPython HTML displaying interface

In [4]:
import os
neuroglancer.server.debug = True
in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ
if in_binder:
    base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], PORT)
    url = 'https://hub.mybinder.org{}'.format(base_prefix)
else:
    url = 'http://localhost:%d' % port
    base_prefix = ''

In [5]:
from IPython.display import HTML

new_url = '{}/v/{}'.format(url, viewer.get_viewer_url().split('/v/')[-1])

HTML(data = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width = "720px" height = "400px" />'.format(url = new_url))